In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,517 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,074 kB]
Hit:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main 

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time
import urllib.request

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('Home_Sales')

In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""SELECT YEAR(date) AS YEAR,
          ROUND(AVG(CASE WHEN bedrooms = 4 THEN price END), 2) AS AVERAGE_PRICE
          FROM Home_Sales
          GROUP BY YEAR""").show(truncate=False)

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|296363.88    |
|2019|300263.7     |
|2020|298353.78    |
|2021|301819.44    |
+----+-------------+



In [6]:
# 4. What is the average price of a home for each year the home was built
# that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""SELECT YEAR(date_built) AS YEAR,
          ROUND(AVG(CASE WHEN bedrooms = 3 AND bathrooms = 3
                    THEN price END), 2) AS AVERAGE_PRICE
          FROM Home_Sales
          GROUP BY YEAR""").show(truncate=False)

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2015|288770.3     |
|2013|295962.27    |
|2014|290852.27    |
|2012|293683.19    |
|2016|290555.07    |
|2010|292859.62    |
|2011|291117.47    |
|2017|292676.79    |
+----+-------------+



In [7]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""SELECT YEAR(date_built) AS YEAR,
          ROUND(AVG(CASE WHEN bedrooms = 3 AND bathrooms = 3
                    AND floors = 2 AND sqft_living >= 2000
                    THEN price END), 2) AS AVERAGE_PRICE
          FROM Home_Sales
          GROUP BY YEAR""").show(truncate=False)

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2015|297609.97    |
|2013|303676.79    |
|2014|298264.72    |
|2012|307539.97    |
|2016|293965.1     |
|2010|285010.22    |
|2011|276553.81    |
|2017|280317.58    |
+----+-------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
start_time = time.time()
spark.sql("""SELECT view,
          ROUND(AVG(CASE WHEN price >= 350000 THEN price ELSE null END), 2) AS AVERAGE_PRICE
          FROM Home_Sales
          GROUP BY view""").show(truncate=False)


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|7   |403005.77    |
|51  |788128.21    |
|15  |404673.3     |
|54  |798684.82    |
|11  |399548.12    |
|29  |397771.65    |
|69  |750537.94    |
|42  |396964.5     |
|87  |1072285.2    |
|73  |752861.18    |
|64  |767036.67    |
|3   |398867.6     |
|30  |397862.0     |
|34  |401419.75    |
|59  |791453.0     |
|8   |398592.71    |
|22  |402022.68    |
|28  |402124.62    |
|85  |1056336.74   |
|16  |399586.53    |
+----+-------------+
only showing top 20 rows

--- 1.0095388889312744 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("""CACHE TABLE Home_Sales""")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached("Home_Sales")

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""SELECT view,
          ROUND(AVG(CASE WHEN price >= 350000 THEN price ELSE null END), 2) AS AVERAGE_PRICE
          FROM Home_Sales
          GROUP BY view""").show(truncate=False)


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|7   |403005.77    |
|51  |788128.21    |
|15  |404673.3     |
|54  |798684.82    |
|11  |399548.12    |
|29  |397771.65    |
|69  |750537.94    |
|42  |396964.5     |
|87  |1072285.2    |
|73  |752861.18    |
|64  |767036.67    |
|3   |398867.6     |
|30  |397862.0     |
|34  |401419.75    |
|59  |791453.0     |
|8   |398592.71    |
|22  |402022.68    |
|28  |402124.62    |
|85  |1056336.74   |
|16  |399586.53    |
+----+-------------+
only showing top 20 rows

--- 0.5513737201690674 seconds ---


In [12]:
# 10. Partition the home sales dataset by date_built field.
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [13]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet("home_sales_partitioned")

In [14]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_homes_p')

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""SELECT view,
          ROUND(AVG(CASE WHEN price >= 350000 THEN price ELSE null END), 2) AS AVERAGE_PRICE
          FROM p_homes_p
          GROUP BY view""").show(truncate=False)


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|7   |403005.77    |
|51  |788128.21    |
|15  |404673.3     |
|54  |798684.82    |
|11  |399548.12    |
|29  |397771.65    |
|69  |750537.94    |
|42  |396964.5     |
|73  |752861.18    |
|87  |1072285.2    |
|64  |767036.67    |
|3   |398867.6     |
|30  |397862.0     |
|34  |401419.75    |
|59  |791453.0     |
|8   |398592.71    |
|22  |402022.68    |
|28  |402124.62    |
|85  |1056336.74   |
|35  |401934.21    |
+----+-------------+
only showing top 20 rows

--- 1.4585306644439697 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table Home_Sales")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('Home_Sales'):
  print('Home_Sales is still cached')
else:
  print('all clear')

all clear
